In [1]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re

In [2]:
data = pd.read_csv('IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [3]:
english_stops = set(stopwords.words('english'))

In [4]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
31527    [this, final, feature, film, michelangelo, ant...
8993     [the, one, thing, said, running, out, of, time...
16069    [i, caught, brief, run, philadelphia, theater,...
12847    [if, done, absolutely, love, film, i, i, mean,...
5587     [it, kind, call, fictionalized, account, barke...
                               ...                        
27450    [unhinged, part, video, nasty, censorship, fil...
16110    [i, find, disconcerting, era, satisfying, fulf...
40964    [a, severe, backwards, step, puppets, mainly, ...
17897    [i, watched, movie, recovering, major, surgery...
1525     [for, movies, life, late, great, chris, farley...
Name: review, Length: 40000, dtype: object 

32925    [i, think, saying, film, many, makes, film, ba...
37878    [i, living, barstow, ca, movie, the, killers, ...
46769    [this, movie, starts, somewhat, slowly, gets, ...
11951    [i, really, looked, forward, see, planet, apes...
18041    [armageddon, ppvthe, last, ppv, smackdown, bra...
 

In [6]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

In [7]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[   8  375  678 ... 4123  613  791]
 [   2    5   65 ...    0    0    0]
 [   1  941 1308 ...    1   47  398]
 ...
 [  39 4500 5650 ... 1669   53  112]
 [   1  197    3 ...    0    0    0]
 [ 206   28   42 ...    0    0    0]] 

Encoded X Test
 [[    1    31   536 ...     0     0     0]
 [    1   504 10653 ...     0     0     0]
 [    8     3   447 ...     0     0     0]
 ...
 [  171    23   120 ...     0     0     0]
 [    1   349  4940 ...  1364  2013   608]
 [    1    41  4785 ...     0     0     0]] 

Maximum review length:  130


In [8]:
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 130, 32)           2955520   
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2,980,417
Trainable params: 2,980,417
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [10]:
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
313/313 [==============================] - ETA: 0s - loss: 0.4432 - accuracy: 0.7758
Epoch 1: accuracy improved from -inf to 0.77585, saving model to models\LSTM.h5
313/313 [==============================] - 111s 298ms/step - loss: 0.4432 - accuracy: 0.7758
Epoch 2/5
313/313 [==============================] - ETA: 0s - loss: 0.2125 - accuracy: 0.9255
Epoch 2: accuracy improved from 0.77585 to 0.92548, saving model to models\LSTM.h5
313/313 [==============================] - 94s 300ms/step - loss: 0.2125 - accuracy: 0.9255
Epoch 3/5
313/313 [==============================] - ETA: 0s - loss: 0.1273 - accuracy: 0.9604
Epoch 3: accuracy improved from 0.92548 to 0.96038, saving model to models\LSTM.h5
313/313 [==============================] - 118s 378ms/step - loss: 0.1273 - accuracy: 0.9604
Epoch 4/5
313/313 [==============================] - ETA: 0s - loss: 0.0813 - accuracy: 0.9775
Epoch 4: accuracy improved from 0.96038 to 0.97747, saving model to models\LSTM.h5
313/313 [====

In [11]:
loaded_model = load_model('models/LSTM.h5')

In [12]:
review = str(input('Movie Review: '))

Movie Review: 49998


In [13]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  
Filtered:  ['']


In [14]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [15]:
result = loaded_model.predict(tokenize_words)
print(result)

1/1 [==============================] - 18s 18s/step
[[0.27414587]]


In [16]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

negative
